# Begining of Basic Image Processing

In [60]:
:dep anyhow = "1.0.65"
:dep opencv = "0.70.0"
:dep plotters = { git = "https://github.com/plotters-rs/plotters", default_features = false, features = ["evcxr", "all_series"] }

use std::fs;
use std::path::{Path};
use anyhow::{anyhow, Result, Error};
use opencv::{self as cv, prelude::*};
use plotters::prelude::*;

let image_dir = "./KU15627-Computer-Vision/images";

In [20]:
for file in fs::read_dir(image_dir).unwrap().filter_map(|file| file.ok()) {
    println!("{}", file.path().to_path_buf().file_name().unwrap().to_str().unwrap());
  }

Balloon.bmp
Airplane.bmp
Earth.bmp
Aerial.bmp
Parrots.bmp
Sailboat.bmp
Pepper.bmp
Lenna.bmp
Mandrill.bmp
Girl.bmp
couple.bmp
milkdrop.bmp


()

In [61]:
let image_path = Path::new(image_dir).join("Mandrill.bmp");
let img: Mat = imgcodecs::imread(image_path.to_str().unwrap(), imgcodecs::IMREAD_UNCHANGED).unwrap();
let root: i64 = BitMapBackend::with_buffer(img, img.mat_size()).into_drawing_area();
root.present()?;


Error: failed to resolve: use of undeclared type `BitMapBackend`

Error: no method named `present` found for type `i64` in the current scope